### Week 7: сделать интерактивное демо, содержащее:

* карты с визуализацией реального и прогнозируемого спроса на такси в выбираемый пользователем момент времени

* временной ряд фактического и прогнозируемого спроса на такси в выбираемой области.

Для реализации демо используем plotly. При реализации динамической карты использовались идеи из
https://github.com/xelibrion/coursera-taxi-vis

In [1]:
from __future__ import (absolute_import, division, print_function, unicode_literals)
import os
import sys

# отключим предупреждения Anaconda
import warnings
warnings.simplefilter('ignore')
warnings.filterwarnings('ignore')

%pylab inline
import pandas as pd
import matplotlib.pyplot as plt
from pandas.tseries.offsets import Hour

print('numpy = ',np.__version__)
print('pandas = ', pd.__version__)
print('Machine: {} {}\n'.format(os.uname().sysname,os.uname().machine))
print(sys.version)

# используем plotly
import plotly.graph_objs as go
import ipywidgets as ipyw
import ipyleaflet as ipyl

from plotly.offline import init_notebook_mode, iplot
from IPython.display import display
from shapely.geometry import Polygon, mapping
import json

Populating the interactive namespace from numpy and matplotlib
numpy =  1.18.1
pandas =  1.0.3
Machine: Darwin x86_64

3.6.10 |Anaconda, Inc.| (default, Jan  7 2020, 15:01:53) 
[GCC 4.2.1 Compatible Clang 4.0.1 (tags/RELEASE_401/final)]


In [2]:
# функция поиска региона по его координатам
def returnRegion(longitude, latitude):
    # Нью-Йорк вписан в прямоугольник от -74.25559 до -73.70001 градусов долготы и от 40.49612 до 40.91553 широты. 
    # нумерация областей идет вертикально верх, начиная с левого нижнего угла
    # 50, 100 .. 2500
    # .. .. .. ..
    # 2 52 .. 2452
    # 1 51 .. 2451    
    # каждая клетка имеет 4 границы: west, east, south, north
    n_longitude = 50
    n_latitude = 50
    longitude_min = -74.25559
    longitude_max = -73.70001
    latitude_min = 40.49612
    latitude_max = 40.91553
    delta_longitude = (longitude_max - longitude_min)/n_longitude
    delta_latitude = (latitude_max - latitude_min)/n_latitude    
    longitude_id = np.abs(np.floor((longitude - longitude_min)/delta_longitude))
    latitude_id = np.abs(np.floor((latitude - latitude_min)/delta_latitude))
    if longitude_id > n_longitude:
        longitude_id = n_longitude
    if latitude_id > n_latitude:
        latitude_id = n_latitude    
    return int(longitude_id * n_longitude + latitude_id + 1)

def getCoordinates(reg_id):
    long1, lat1, long2, lat2 = regions_df[regions_df['region'] == reg_id].iloc[0][['west', 'north', 'east', 'south']]
    return 0.5*(long1 + long2), 0.5*(lat1 + lat2)

# Загрузим данные

In [4]:
# загрузим список интересуемых ячеек (102 ячейки, данные сохранены на 2-ой неделе)
regions = pd.read_csv('./102_regions.csv')
regions = list(map(int, regions))
R = len(regions) # количество рядов 102
# загрузим идентификаторы ячеек из файла regions.csv
regions_df = pd.read_csv('./regions.csv', sep = ';')
#regions_df.set_index('region', inplace = True)
#regions_df = regions_df.loc[regions]

forecast_path = './data/'

# загружаем прогноз на 1 час вперед
df_pred = pd.read_hdf(forecast_path + 'k_' + str(1) + '.h5') 
df_pred.columns = df_pred.columns.astype(int)

# загружаем точные значения
df_y = pd.read_hdf('./true_values.h5')
df_y.sort_index(axis=0, ascending=True, inplace=True)
df_y.columns = df_y.columns.astype(int)

# получим список регионов и их координаты (возьмем центр каждой ячейки)
regions_coordinates_df = pd.DataFrame(list(map(getCoordinates, regions)))
regions_coordinates_df.index = regions
regions_coordinates_df.columns = ['longitude', 'latitude']
regs_df = pd.DataFrame(data = {'longitude': regions_coordinates_df['longitude'], \
                               'latitude': regions_coordinates_df['latitude']}, index = regions)

index = pd.date_range('2016-06-01', '2016-06-30 17:00', freq='H')

# временной ряд фактического и прогнозируемого спроса на такси в выбираемой области

In [5]:
init_notebook_mode()

def period_obtain(date):    
    start = pd.to_datetime(date) - Hour(84)    
    if start < index.min():
        start, end = index.min(), index.min() + Hour(168)
    else:
        end = pd.to_datetime(date) + Hour(83)
        if end > index.max():
            start, end = index.max() - Hour(168), index.max()    
    return slice(str(start), str(end))

def draw_picts(region, date):
    plot_output.clear_output()

    with plot_output:
        date_idx = period_obtain(date)
        data_f, data_y = df_pred[(region)].loc[date_idx], df_y[(region)].loc[date_idx]        
        d1 = go.Scatter(x=data_y.index, y=data_y, mode='lines+markers', name='real',)
        d2 = go.Scatter(x=data_f.index, y=data_f, mode='lines+markers', name='1h prediction',)

        data = [d1, d2]
        layout = go.Layout(title='Real vs 1h prediction', width=800, height=640)
        fig = go.Figure(data=data, layout=layout)
        iplot(data, show_link=False)
        
def region_ch(change):
    region = change['new']
    draw_picts(region, date_slider.value)

def date_ch(change):
    field = change['new']
    draw_picts(region_slider.value, field)

region_slider = ipyw.SelectionSlider(options=list(regs_df.index), description=u'Регион:', disabled=False,
                                     continuous_update=False, orientation='horizontal', readout=True)

date_slider = ipyw.SelectionSlider(options=list(pd.date_range('2016-06-01', '2016-06-30').astype(str)),
                                   description=u'Дата:', disabled=False, continuous_update=False,
                                   orientation='horizontal', readout=True)
display(region_slider)
display(date_slider)

plot_output = ipyw.Output()
display(plot_output)

region_slider.observe(region_ch, names='value')
date_slider.observe(date_ch, names='value')

draw_picts(region_slider.value, date_slider.value)

SelectionSlider(continuous_update=False, description='Регион:', options=(1075, 1076, 1077, 1125, 1126, 1127, 1…

SelectionSlider(continuous_update=False, description='Дата:', options=('2016-06-01', '2016-06-02', '2016-06-03…

Output()

# карты с визуализацией реального и прогнозируемого спроса на такси в выбираемый пользователем момент времени

In [6]:
# переформатируем данные

regions_df.set_index('region', inplace = True)
regions_df = regions_df.loc[regions]

tmp = pd.DataFrame()
for c in df_pred.columns:
    tmp = tmp.append(pd.DataFrame(data = {'1h': df_pred[c]}).join(pd.DataFrame(data = {'region': [c] * len(df_pred[c])}, index = df_pred[c].index)))
df_pred = tmp.reset_index().set_index(['region', 'index'])

tmp = pd.DataFrame()
for c in df_y.columns:
    tmp = tmp.append(pd.DataFrame(data = {'1h': df_y[c]}).join(pd.DataFrame(data = {'region': [c] * len(df_y[c])}, index = df_y[c].index)))
df_y = tmp.reset_index().set_index(['region', 'index'])

del tmp

In [7]:
def region_to_geofeature(row):
    def get_color(error_group):
        colormap = {0: '#ffffb2', 1: '#fecc5c', 2: '#fd8d3c', 3: '#f03b20', 4: '#bd0026', }
        return colormap[error_group]
    lats = row[['north', 'south', 'south', 'north']].values
    lons = row[['west', 'west', 'east', 'east']].values
    coords = zip(lons, lats)
    poly = Polygon(coords)
    return {
        'type': 'Feature', 'id': row.name, 'geometry': mapping(poly),
        'properties': {'style': {'color': 'grey', 'weight': 1, 'fillColor': get_color(row['error_group']), 'fillOpacity': 0.6}}
    }

def get_geo_data(timestamp):
    regions_to_draw = mean_errors.loc[timestamp].join(regions_df)
    regions_to_draw.index = regions_to_draw.index.droplevel(0)
    regions_to_draw.loc[:, 'error_group'] = pd.cut(regions_to_draw['mean_error'], 5, labels=False)
    geo_features = regions_to_draw.apply(region_to_geofeature, axis=1).values
    return {'type': 'FeatureCollection', 'features': [x for x in geo_features]}

def region_stats(region, timestamp):
    forecast = round(df_pred.loc[region].loc[timestamp], 1)
    y = df_y.loc[region].loc[timestamp]
    errors = y - forecast
    errors_percents = round(100*abs(errors)/forecast, 1)
    errors.index = forecast.index
    df = pd.DataFrame(
        {
            'Forecast': forecast,
            'True value': y,
            'Error': errors, 
            'Error_pct': errors_percents
        },
        index=df_pred.columns)
    df = df[['Forecast', 'True value', 'Error', 'Error_pct']]
    return df

In [8]:
# будем отображать цветом уровень абсолютной ошибки в прогнозе
mean_errors = (abs(df_pred - df_y)).reset_index().set_index(['index', 'region'])
mean_errors.sort_index(inplace=True)
mean_errors.columns = ['mean_error']

# итерактивность
esb_lat, esb_lon = 40.7484, -73.9857

hours = pd.date_range('2016-06-01 00:00', '2016-06-30 17:00', freq='H')
hours_list = list(hours.astype(str))
map_date_selector = ipyw.SelectionSlider(
    options=hours_list, description=u'Выберите дату:', continuous_update=False, orientation='horizontal',
    readout=True, layout=ipyw.Layout(width='100%'))

map_region_label = ipyw.Label(description=u'Регион:')

tbl_region_info = ipyw.HTML()
tbl_region_info.add_class('rendered_html')

map = ipyl.Map(center=[esb_lat, esb_lon], zoom=11, min_zoom=10, layout=ipyw.Layout(height='500px', align_self='stretch'), )

def hover_handler(event=None, id=None, properties=None):
    map_region_label.value = str(id)
    tbl_region_info.value = region_stats(id, map_date_selector.value).to_html()

def draw_regions(timestamp):
    json_layer = [x for x in map.layers if isinstance(x, ipyl.GeoJSON)]
    if json_layer:
        map.remove_layer(json_layer[0])

    geo_data = get_geo_data(timestamp)
    layer = ipyl.GeoJSON(data=geo_data, hover_style={'fillColor': 'blue'})
    layer.on_hover(hover_handler)
    map.add_layer(layer)

def map_on_date_change(change):
    date = change['new']
    draw_regions(date)

map_date_selector.observe(map_on_date_change, names='value')

In [9]:
# ниже в динамие будет отображаться прогнозное значение, реальное и ошибки прогноза
# для смены региона достаточно навести указатель мышки на другой регион в динамической карте, отображаемой
# в следующей ячейке
display(map_region_label)
display(tbl_region_info)

Label(value='', description='Регион:')

HTML(value='', _dom_classes=('rendered_html',))

In [9]:
# отображаем карту с зонами, подсвеченными по уровням абсолютной ошибки в прогнозе
display(map_date_selector)
display(map)

draw_regions(map_date_selector.value)

SelectionSlider(continuous_update=False, description='Выберите дату:', layout=Layout(width='100%'), options=('2016-06-01 00:00:00', '2016-06-01 01:00:00', '2016-06-01 02:00:00', '2016-06-01 03:00:00', '2016-06-01 04:00:00', '2016-06-01 05:00:00', '2016-06-01 06:00:00', '2016-06-01 07:00:00', '2016-06-01 08:00:00', '2016-06-01 09:00:00', '2016-06-01 10:00:00', '2016-06-01 11:00:00', '2016-06-01 12:00:00', '2016-06-01 13:00:00', '2016-06-01 14:00:00', '2016-06-01 15:00:00', '2016-06-01 16:00:00', '2016-06-01 17:00:00', '2016-06-01 18:00:00', '2016-06-01 19:00:00', '2016-06-01 20:00:00', '2016-06-01 21:00:00', '2016-06-01 22:00:00', '2016-06-01 23:00:00', '2016-06-02 00:00:00', '2016-06-02 01:00:00', '2016-06-02 02:00:00', '2016-06-02 03:00:00', '2016-06-02 04:00:00', '2016-06-02 05:00:00', '2016-06-02 06:00:00', '2016-06-02 07:00:00', '2016-06-02 08:00:00', '2016-06-02 09:00:00', '2016-06-02 10:00:00', '2016-06-02 11:00:00', '2016-06-02 12:00:00', '2016-06-02 13:00:00', '2016-06-02 14:00:00', '2016-06-02 15:00:00', '2016-06-02 16:00:00', '2016-06-02 17:00:00', '2016-06-02 18:00:00', '2016-06-02 19:00:00', '2016-06-02 20:00:00', '2016-06-02 21:00:00', '2016-06-02 22:00:00', '2016-06-02 23:00:00', '2016-06-03 00:00:00', '2016-06-03 01:00:00', '2016-06-03 02:00:00', '2016-06-03 03:00:00', '2016-06-03 04:00:00', '2016-06-03 05:00:00', '2016-06-03 06:00:00', '2016-06-03 07:00:00', '2016-06-03 08:00:00', '2016-06-03 09:00:00', '2016-06-03 10:00:00', '2016-06-03 11:00:00', '2016-06-03 12:00:00', '2016-06-03 13:00:00', '2016-06-03 14:00:00', '2016-06-03 15:00:00', '2016-06-03 16:00:00', '2016-06-03 17:00:00', '2016-06-03 18:00:00', '2016-06-03 19:00:00', '2016-06-03 20:00:00', '2016-06-03 21:00:00', '2016-06-03 22:00:00', '2016-06-03 23:00:00', '2016-06-04 00:00:00', '2016-06-04 01:00:00', '2016-06-04 02:00:00', '2016-06-04 03:00:00', '2016-06-04 04:00:00', '2016-06-04 05:00:00', '2016-06-04 06:00:00', '2016-06-04 07:00:00', '2016-06-04 08:00:00', '2016-06-04 09:00:00', '2016-06-04 10:00:00', '2016-06-04 11:00:00', '2016-06-04 12:00:00', '2016-06-04 13:00:00', '2016-06-04 14:00:00', '2016-06-04 15:00:00', '2016-06-04 16:00:00', '2016-06-04 17:00:00', '2016-06-04 18:00:00', '2016-06-04 19:00:00', '2016-06-04 20:00:00', '2016-06-04 21:00:00', '2016-06-04 22:00:00', '2016-06-04 23:00:00', '2016-06-05 00:00:00', '2016-06-05 01:00:00', '2016-06-05 02:00:00', '2016-06-05 03:00:00', '2016-06-05 04:00:00', '2016-06-05 05:00:00', '2016-06-05 06:00:00', '2016-06-05 07:00:00', '2016-06-05 08:00:00', '2016-06-05 09:00:00', '2016-06-05 10:00:00', '2016-06-05 11:00:00', '2016-06-05 12:00:00', '2016-06-05 13:00:00', '2016-06-05 14:00:00', '2016-06-05 15:00:00', '2016-06-05 16:00:00', '2016-06-05 17:00:00', '2016-06-05 18:00:00', '2016-06-05 19:00:00', '2016-06-05 20:00:00', '2016-06-05 21:00:00', '2016-06-05 22:00:00', '2016-06-05 23:00:00', '2016-06-06 00:00:00', '2016-06-06 01:00:00', '2016-06-06 02:00:00', '2016-06-06 03:00:00', '2016-06-06 04:00:00', '2016-06-06 05:00:00', '2016-06-06 06:00:00', '2016-06-06 07:00:00', '2016-06-06 08:00:00', '2016-06-06 09:00:00', '2016-06-06 10:00:00', '2016-06-06 11:00:00', '2016-06-06 12:00:00', '2016-06-06 13:00:00', '2016-06-06 14:00:00', '2016-06-06 15:00:00', '2016-06-06 16:00:00', '2016-06-06 17:00:00', '2016-06-06 18:00:00', '2016-06-06 19:00:00', '2016-06-06 20:00:00', '2016-06-06 21:00:00', '2016-06-06 22:00:00', '2016-06-06 23:00:00', '2016-06-07 00:00:00', '2016-06-07 01:00:00', '2016-06-07 02:00:00', '2016-06-07 03:00:00', '2016-06-07 04:00:00', '2016-06-07 05:00:00', '2016-06-07 06:00:00', '2016-06-07 07:00:00', '2016-06-07 08:00:00', '2016-06-07 09:00:00', '2016-06-07 10:00:00', '2016-06-07 11:00:00', '2016-06-07 12:00:00', '2016-06-07 13:00:00', '2016-06-07 14:00:00', '2016-06-07 15:00:00', '2016-06-07 16:00:00', '2016-06-07 17:00:00', '2016-06-07 18:00:00', '2016-06-07 19:00:00', '2016-06-07 20:00:00', '2016-06-07 21:00:00', '2016-06-07 22:00:00', '2016-06-07 23:00:00', '2016-06-08 00:00:00', '201

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map data (c) <a href="https://openstreetmap.org">OpenStreetMap</a> contributors'}, center=[40.7484, -73.9857], default_tiles=TileLayer(max_zoom=19, min_zoom=1, options=['attribution', 'detect_retina', 'max_zoom', 'min_zoom', 'opacity', 'tile_size']), layers=(TileLayer(max_zoom=19, min_zoom=1, options=['attribution', 'detect_retina', 'max_zoom', 'min_zoom', 'opacity', 'tile_size']),), layout=Layout(align_self='stretch', height='500px'), min_zoom=10, options=['attribution_control', 'basemap', 'bounce_at_zoom_limits', 'box_zoom', 'center', 'close_popup_on_click', 'double_click_zoom', 'dragging', 'inertia', 'inertia_deceleration', 'inertia_max_speed', 'keyboard', 'keyboard_pan_offset', 'keyboard_zoom_offset', 'max_zoom', 'min_zoom', 'scroll_wheel_zoom', 'tap', 'tap_tolerance', 'touch_zoom', 'world_copy_jump', 'zoom', 'zoom_animation_threshold', 'zoom_control', 'zoom_start'], zoom=11)